In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import pandas as pd
import torch
!pip install transformers
from transformers import BertTokenizer
# from keras.preprocessing.sequence import pad_sequences
from torch.nn import functional as F

     |████████████████████████████████| 542kB 6.9MB/s 
     |████████████████████████████████| 3.7MB 41.5MB/s 
     |████████████████████████████████| 870kB 46.5MB/s 
     |████████████████████████████████| 1.0MB 42.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=549e696df39ae9c43703b2c9f14cb9686f469f4f01781582072f942a76a1308b
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [0]:
import pickle
with open('/content/drive/My Drive/data.pkl','rb') as f:
  df=pickle.load(f)

In [0]:
MAX_LENGTH=len(df.iloc[0]['encoding'])
tag_index={'[PAD]':0,'<s>':1,'B-AG':2,'I-AG':3,'B-TG':4,'I-TG':5,'O':6,'</s>':7}
df['labels']=df['labels'].apply(lambda x: [tag_index[lbl] for lbl in x])

In [0]:
from torch.utils.data import Dataset
class SequenceDataset(Dataset):
  def __init__(self,df):
    self.df=df

  def __len__(self):
    return len(self.df)

  def __getitem__(self,index):
    return torch.tensor(self.df.iloc[index]['encoding']),torch.tensor(self.df.iloc[index]['attn_mask']),torch.tensor(self.df.iloc[index]['labels'])

In [0]:
from torch.utils.data import DataLoader
import numpy as np
msk = np.random.rand(len(df)) < 0.8
train=df[msk]
val=df[~msk]
train_set=SequenceDataset(train)
val_set=SequenceDataset(val)
train_loader=DataLoader(train_set, batch_size = 2)
val_loader = DataLoader(val_set, batch_size = 2)

In [0]:
import torch.nn as nn
from transformers import BertModel
class Encoder(nn.Module):
    def __init__(self, freeze_bert = True):
        super(Encoder, self).__init__()
        self.bert_layer = BertModel.from_pretrained('bert-base-uncased')

        # for p in self.bert_layer.parameters():
        #     p.requires_grad = False

    def forward(self, seq, attn_masks):
        cont_reps, _ = self.bert_layer(seq, attention_mask = attn_masks)
        return cont_reps[:,0]        

In [0]:
class Decoder(nn.Module):
    def __init__(self,vocab_size, hidden_size,output_size, dropout_p=0.1):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.vocab_size=vocab_size
        self.output_size = output_size
        self.dropout = nn.Dropout(0.1)
        self.embedding = nn.Embedding(self.vocab_size,self.hidden_size)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size,batch_first=True)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, hidden,input):
       embedded = self.embedding(input).view(hidden.shape[0], 1, -1)
       embedded = self.dropout(embedded)
       output, hidden = self.gru(embedded, hidden.permute(1,0,2).contiguous())
       output = F.log_softmax(self.out(output), dim=2)
       return output, hidden.permute(1,0,2)   

In [0]:
import torch.optim as optim
import tqdm
import os
import pickle
# embedding_size=100
hidden_size=768
# bert_model=BertModel.from_pretrained('bert-base-uncased')
vocab_size=len(tag_index)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
encoder=Encoder().to(device)

if(os.path.exists('/content/drive/My Drive/BERT-SEQ-Tagger/encoder.pt')):
    encoder.load_state_dict(torch.load('/content/drive/My Drive/BERT-SEQ-Tagger/encoder.pt'))

decoder=Decoder(vocab_size,hidden_size,len(tag_index)).to(device)
if(os.path.exists('/content/drive/My Drive/BERT-SEQ-Tagger/decoder.pt')):
    decoder.load_state_dict(torch.load('/content/drive/My Drive/BERT-SEQ-Tagger/decoder.pt'))

criterion = nn.NLLLoss(ignore_index=tag_index['[PAD]'])
# enc_optimizer = optim.Adam(encoder.parameters(), lr = 2e-5)
dec_optimizer = optim.Adam(decoder.parameters(), lr = 2e-5)

# training_loss=[]
val_losses=[]
if(os.path.exists('/content/drive/My Drive/BERT-SEQ-Tagger/val_losses.pkl')):
  with open('/content/drive/My Drive/BERT-SEQ-Tagger/val_losses.pkl','rb') as f:
    val_losses=pickle.load(f)

for _e in range(200):
    train_loss=0
    for t, (seq, attn_mask, labels) in enumerate(train_loader):
        # data_batch = sort_batch_by_len(data_dict)
        batch_size=seq.shape[0]
        seq=seq.to(device)
        attn_mask=attn_mask.to(device)
        labels =labels.to(device) #torch.tensor(data_batch).to(device)
                
        # enc_optimizer.zero_grad()
        dec_optimizer.zero_grad()
        encoder_output=encoder(seq,attn_mask)        
        decoder_input = torch.tensor([batch_size*[tag_index['<s>']]], device=device).view(-1,1)
        decoder_hidden=encoder_output.view(batch_size,1,-1)
        labels= torch.cat((labels,torch.tensor(batch_size*[tag_index['</s>']], device=device).view(-1,1)),dim=1)
        loss=0
        for di in range(labels.shape[1]):
          decoder_output,decoder_hidden=decoder(decoder_hidden,decoder_input)
          # print(decoder_output.squeeze(0).shape)
          loss += criterion(decoder_output.view(encoder_output.shape[0],-1), labels[:,di])
          train_loss+=loss.data.item()
          decoder_input = labels[:,di]
        loss.backward()   
        # enc_optimizer.step()
        dec_optimizer.step()
    train_loss=train_loss/len(train)
    # print(train_loss)    
    val_loss=0
    for t, (seq, attn_mask, labels) in enumerate(val_loader):
        seq=seq.to(device)
        attn_mask=attn_mask.to(device)
        labels =labels.to(device) #torch.tensor(data_batch).to(device)
        batch_size=seq.shape[0]
        # enc_optimizer.zero_grad()
        dec_optimizer.zero_grad()
        encoder_output=encoder(seq,attn_mask)
        decoder_input = torch.tensor([batch_size*[tag_index['<s>']]], device=device).view(-1,1)
        decoder_hidden=encoder_output.view(batch_size,1,-1)
        labels= torch.cat((labels,torch.tensor(batch_size*[tag_index['</s>']], device=device).view(-1,1)),dim=1)
        loss=0
        for di in range(labels.shape[1]):
          decoder_output,decoder_hidden=decoder(decoder_hidden,decoder_input)
          # print(decoder_output.squeeze(0).shape)
          loss += criterion(decoder_output.view(encoder_output.shape[0],-1), labels[:,di])
          decoder_input = labels[:,di]
          # _, top_idx = decoder_output.data.topk(1)
          # decoder_input = top_idx.view(-1)
          val_loss+=loss.data.item()
    val_loss=val_loss/len(val)
    if(len(val_losses)>0 and val_loss<min(val_losses)):
      torch.save(encoder.state_dict(), '/content/drive/My Drive/BERT-SEQ-Tagger/encoder.pt') 
      torch.save(decoder.state_dict(), '/content/drive/My Drive/BERT-SEQ-Tagger/decoder.pt')  
    val_losses.append(val_loss)      
    print('training loss:{} validation loss:{}'.format(train_loss,val_loss))       

training loss:356.19106928213375 validation loss:392.052931886267
training loss:354.71986450172784 validation loss:392.6133257577548
training loss:354.022231712867 validation loss:392.12853471572333
training loss:353.2969987895545 validation loss:391.92049436875294
training loss:352.2381025736726 validation loss:393.54688311187
training loss:351.4868782266857 validation loss:392.4445551754655
training loss:351.39680223953064 validation loss:392.4150961113943
training loss:350.98593977154707 validation loss:392.18388430250656
training loss:350.489466122004 validation loss:390.9464576534323
training loss:350.08104702318747 validation loss:392.9434012239044
training loss:349.7249166811545 validation loss:392.30960919405965
training loss:349.28548282018795 validation loss:390.9471149605674
training loss:348.68554939454 validation loss:393.1141022556537
training loss:348.21755079772527 validation loss:392.4692919528162
training loss:348.3713643511449 validation loss:392.43028806754063
train

In [0]:
import pickle
with open('/content/drive/My Drive/BERT-SEQ-Tagger/val_losses.pkl','wb') as f:
  pickle.dump(val_losses,f)